# **Настройки + библиотеки**

In [ ]:
#Установка catboost
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.9 MB/s eta 0:00:00


In [ ]:
# библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier
%matplotlib inline
# подключение диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# датасет
dataset_path = '/content/drive/My Drive/Bases/WIND_DBASE/dag_2011_2022_final.csv'

# интесесущая станция
st_name = 'Кочубей'

# интересующие признаки данных
col_list = ['datetime',
            't_air', 't_soil', 'P_atm', 'wind_dir', 'wind_speed']

# границы обучающей выборки
train_borders = ['2011-01-01 00:00:00', '2020-12-31 21:00:00']
# границы проверочной выборки
val_borders = ['2021-01-01 00:00:00', '2021-12-31 21:00:00']
# границы тестовой выборки
test_borders = ['2022-01-01 00:00:00', '2022-12-31 21:00:00']

# **Функции**

**Общие функции**

In [ ]:
# подготовка датасета (чтение, выделение признаков, нормировка, класс ветра)
# path - путь к исходнику
# st_name - название станции
# col_list - колонки признаков
# mode = 'category' / 'values'
def process_dataset(path, st_name, col_list, norm_0_1=False):
  # читаем данные
  df = pd.read_csv(path)
  # выделяем станцию
  df = df[df.st_name == st_name]
  # выделяем признаки
  df = df[col_list]
  # перевод градусов угла направления ветра в индекс лепестка розы ветров [0..7]
  def conv_360_to_8(val):
    if val >= 360:
      return 0
    else:
      return val // 45
    return res
  def conv_360_to_16(val):
    if val >= 360:
      return 0
    else:
      return val // 22.5
    return res
  df['wind_dir'] = df['wind_dir'].apply(conv_360_to_16).astype('int32')

  # округление температуры воздуха
  df['t_air'] = df['t_air'].astype('int32')
  # приведение температуры воздуха к категориям от 0
  df['t_air'] = df['t_air'] - df['t_air'].min()

  # округление температуры почвы
  df['t_soil'] = df['t_soil'].astype('int32')
  # приведение температуры воздуха к категориям от 0
  df['t_soil'] = df['t_soil'] - df['t_soil'].min()

  # округление атмосферного давления
  df['P_atm'] = df['P_atm'].astype('int32')
  # приведение температуры воздуха к категориям от 0
  df['P_atm'] = df['P_atm'] - df['P_atm'].min()

  # приводим скорости ветра к 3 классам:
  # 0 - от 0 до 3 м/с
  # 1 - от 4 до 7 м/с
  # 2 - от 8 м/с и выше
  def wind_to_class(x):
    if x < 4:
      res = 0
    elif x < 8:
      res = 1
    else:
      res = 2
    return res
  df['wind_class'] = df['wind_speed'].apply(wind_to_class).astype('int32')

  if norm_0_1:
    # нормировка столбцов [0..1]
    for col in col_list[1:]:
      a = df[col].min()
      b = df[col].max()
      df[col] = (df[col] - a)/(b - a)
      df[col] = df[col].astype('float32')

  return df

# нарезка примеров из датафрейма
# df - исходный датафрейм
# days - размер блока данных для X примера (по умолчанию 1 день)
# col_list - колонки c которыми делается выборка
# borders - список из двух значений - границ выборки
def create_samples(df, col_list, borders, days_count=1):
  # выделение данных по границам времени
  rab_df = df[(df.datetime >= borders[0]) & (df.datetime <= borders[1])]
  # размер блока данных
  block_size = days_count * 8
  # начальная позиция текущего смещения в датасете
  pos = 0
  # обнуляем списки примеров...
  samples_x = []
  samples_y = []
  # флаг остановки цикла
  Ex = False
  # нарезка данных в цикле
  while not Ex:
    # если не дошли до конца....
    if pos + block_size < rab_df.shape[0]:
      # формирование вектора X для всех колонок
      x_vec = []
      for col in col_list:
        x_vec.extend(rab_df[col][pos:pos+block_size].to_list())
      # формирование вектора Y
      y_vec = rab_df['wind_class'][pos+block_size:pos+block_size+1].to_list()[0]
      # добавление векторов в списки примеров
      samples_x.append(x_vec)
      samples_y.append(y_vec)
    else:
      # прекращение нарезки когда доходим до конца
      Ex = True
    # сдвиг позиции нарезки
    pos += 1 # сдвигаем позицию на 1
  # результат в numpy массивах
  return np.array(samples_x), np.array(samples_y)

# получение весов классов
# df - исходный датафрейм
# borders - список из двух значений - границ выборки
def get_class_weights(df, borders, class_num=3):
  rab_df = df[(df.datetime >= borders[0]) & (df.datetime <= borders[1])]['wind_class']
  class_weights = {}
  for i in range(class_num):
    class_weights[i] = 1.0 / rab_df.value_counts()[i]
  return class_weights

# на основе проверочной или тестовой выборки
# возвращает:
# 1. Точность предсказания
# 2. Список точностей для классов
# 3. Матрицу несоответствий
def get_accuracy_report(model, x, y, class_num=3):
  # предикт
  y_pred = model.predict(x)
  # numpy массив истинных y_true
  y_true = y
  # матрица несоответсвий
  cm = confusion_matrix(y_true, y_pred)
  res = [None for i in range(class_num)]
  for i in range(class_num):
    res[i] = round(cm[i, i]/(cm[i, :].sum()),3)
  return accuracy_score(y_true, y_pred), res, cm


# **Чтение и обработка данных, подготовка примеров**

In [ ]:
#загрузка и предобработка данных
df = process_dataset(dataset_path, st_name, col_list)
df

,datetime,t_air,t_soil,P_atm,wind_dir,wind_speed,wind_class
2,2011-01-01 00:00:00,33,34,26,15,3,0
6,2011-01-01 03:00:00,33,33,25,15,3,0
9,2011-01-01 06:00:00,33,33,27,13,3,0
14,2011-01-01 09:00:00,32,34,27,13,4,1
17,2011-01-01 12:00:00,31,32,27,13,3,0
...,...,...,...,...,...,...,...
139988,2022-12-31 09:00:00,33,38,38,0,0,0
139993,2022-12-31 12:00:00,36,37,37,6,1,0
139998,2022-12-31 15:00:00,32,28,37,0,0,0
140000,2022-12-31 18:00:00,29,28,42,0,0,0


In [ ]:
# количество дней в блоке данных
days_count = 1
# нарезка примеров для обучения
train_x, train_y = create_samples(df, col_list[1:], train_borders, days_count=days_count)
# нарезка примеров для проверки
val_x, val_y = create_samples(df, col_list[1:], val_borders, days_count=days_count)
# нарезка примеров для теста
test_x, test_y = create_samples(df, col_list[1:], test_borders, days_count=days_count)
# размеры выборки
print('Обучающая:', train_x.shape,  train_y.shape)
print('Проверочная:', val_x.shape,  val_y.shape)
print('Тестовая:', test_x.shape,  test_y.shape)

# настройка весов классов для обучающей выборки
class_weight = get_class_weights(df, train_borders)
print(class_weight)

Обучающая: (29216, 40) (29216,)
Проверочная: (2912, 40) (2912,)
Тестовая: (2912, 40) (2912,)
{0: 6.947821857847565e-05, 1: 8.482483671218933e-05, 2: 0.0003287310979618672}


# **ОБУЧЕНИЕ МОДЕЛЕЙ**

In [ ]:
# Объявление CatBoostClassifier и обучение
model = CatBoostClassifier(iterations = 1000, class_weights=class_weight)
model.fit(train_x, train_y)

Learning rate set to 0.093987
0:	learn: 1.0387157	total: 33.5ms	remaining: 33.5s
1:	learn: 0.9906757	total: 60.5ms	remaining: 30.2s
2:	learn: 0.9508275	total: 89.5ms	remaining: 29.8s
3:	learn: 0.9179057	total: 118ms	remaining: 29.4s
4:	learn: 0.8895493	total: 144ms	remaining: 28.8s
5:	learn: 0.8654416	total: 174ms	remaining: 28.9s
6:	learn: 0.8441945	total: 205ms	remaining: 29s
7:	learn: 0.8261747	total: 237ms	remaining: 29.4s
8:	learn: 0.8102561	total: 264ms	remaining: 29s
9:	learn: 0.7967131	total: 290ms	remaining: 28.7s
10:	learn: 0.7843151	total: 318ms	remaining: 28.6s
11:	learn: 0.7732138	total: 350ms	remaining: 28.9s
12:	learn: 0.7631377	total: 382ms	remaining: 29s
13:	learn: 0.7546223	total: 413ms	remaining: 29.1s
14:	learn: 0.7472916	total: 439ms	remaining: 28.8s
15:	learn: 0.7396713	total: 467ms	remaining: 28.7s
16:	learn: 0.7333054	total: 493ms	remaining: 28.5s
17:	learn: 0.7272632	total: 520ms	remaining: 28.4s
18:	learn: 0.7220851	total: 557ms	remaining: 28.8s
19:	learn: 0.7

In [ ]:
# Проверочная выборка
total_accuracy, accuracies, cm = get_accuracy_report(model, val_x, val_y)
for index, value in enumerate(accuracies):
  print(f"Точность класса {index}: {round(value*100, 2)} %")
print(cm)
print(f"Общая точность: {total_accuracy}")

Точность класса 0: 81.2 %
Точность класса 1: 62.3 %
Точность класса 2: 68.3 %
[[1236  270   17]
 [ 296  659  103]
 [  17   88  226]]
Общая точность: 0.7283653846153846


In [ ]:
# Тестовая выборка
total_accuracy, accuracies, cm = get_accuracy_report(model, test_x, test_y)
for index, value in enumerate(accuracies):
  print(f"Точность класса {index}: {round(value*100, 2)} %")
print(cm)
print(f"Общая точность: {total_accuracy}")

Точность класса 0: 89.9 %
Точность класса 1: 62.9 %
Точность класса 2: 60.0 %
[[2284  257    1]
 [ 113  220   17]
 [   1    7   12]]
Общая точность: 0.864010989010989
